In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score 
from nltk.stem.porter import PorterStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer

About The Dataset:


1. Id: unique id for a news article
2. Title: the title of a news article
3. Author: author of the news article
4. Text: the text of the article; could be incomplete
5. Label: a label that marks whether the news article is real or fake:

           1: Fake News
           0: Real News

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Stopwords - Un-usefull words which doesn't adds meaning to our data
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [5]:
df = pd.read_csv("D:\\Python.vs\\Data Set\\news_data.csv")

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [8]:
df.shape

(20800, 5)

In [9]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
# Replacing the null values with empty string

df = df.fillna(" ")

In [11]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [12]:
# Merging the author name and news title

df["content"] = df["author"] +" "+ df["title"]

In [13]:
df["content"]

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [14]:
# Separating the data & label
X = df.drop(columns = "label", axis = 1)
Y = df["label"]

In [15]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

Stemming is the process of reducing a word to its Root word

Example:
actor, actress, acting --> act

In [16]:
port_stem = PorterStemmer()

In [17]:
def stemming(content):
    # Reducing everything which is not a Alphabate
    stemmed_content = re.sub("[^a-zA-Z]"," ",content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content

In [18]:
df['content'] = df['content'].apply(stemming)

In [19]:
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [20]:
# Separating the data and label
X = df["content"].values
Y = df["label"].values

In [21]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [22]:
print(X.shape)
print(Y.shape)

(20800,)
(20800,)


In [23]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 112)

In [25]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [26]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9864182692307693


In [27]:
# Accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9810096153846154


In [28]:
from sklearn import metrics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc, accuracy_score

In [29]:
def Model(data , models, thrsold, grid = False, cv = False):
    X = data["content"].values
    Y = data["label"].values

    # Converting the textual data to numerical data
    vectorizer = TfidfVectorizer()
    vectorizer.fit(X)
    X = vectorizer.transform(X)

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25,  stratify = Y, random_state = 112)

    col_names = ["Algo", "Accuracy", "Precision", "Recall", "F1_score", "Auc"]
    results_train = pd.DataFrame(columns = col_names)
    results_test = pd.DataFrame(columns = col_names)


    i = 0
    for name, model in models:

        model.fit(x_train, y_train)
        x_train_pred = model.predict_proba(x_train)[:,1]
        accuracy, precision, recall, f1_score, roc_auc = eval_metrics_train(y_train, x_train_pred, thrsold)
        results_train.loc[i] = [name,accuracy,precision,recall,f1_score,roc_auc]
        
        x_test_pred = model.predict_proba(x_test)[:,1]
        accuracy, precision, recall, f1_score, roc_auc = eval_metrics_test(y_test, x_test_pred, thrsold)
        results_test.loc[i] = [name,accuracy,precision,recall,f1_score,roc_auc]
        i += 1
    
    return(results_train, results_test)

In [30]:
def eval_metrics_test(target, prob, threshold):
        """ target = y_yest,prob = logistic.predict(X_test),Threshold = Cutoff Threshold """
        data_1=pd.DataFrame()
        data_1['pred_proba'] = pd.Series(prob)
        data_1['pred'] = data_1['pred_proba'].map(lambda x: 1.0 if x > threshold else 0.0)
        accuracy=metrics.accuracy_score(target,data_1['pred'])
        precision= metrics.precision_score(target,data_1['pred'])
        recall= metrics.recall_score(target,data_1['pred'])
        f1_score = metrics.f1_score(target,data_1['pred'])
        roc_auc = metrics.roc_auc_score(target,data_1['pred'])
        return(accuracy,precision,recall,f1_score,roc_auc)

In [31]:
def eval_metrics_train(target, prob, threshold):
        """ target = y_yest,prob = logistic.predict(X_test),Threshold = Cutoff Threshold """
        data=pd.DataFrame()
        data['pred_proba'] = pd.Series(prob)
        data['pred'] = data['pred_proba'].map(lambda x: 1.0 if x > threshold else 0.0)
        accuracy=metrics.accuracy_score(target,data['pred'])
        precision= metrics.precision_score(target,data['pred'])
        recall= metrics.recall_score(target,data['pred'])
        f1_score = metrics.f1_score(target,data['pred'])
        roc_auc = metrics.roc_auc_score(target,data['pred'])
        return(accuracy,precision,recall,f1_score,roc_auc)

In [38]:
models = []
models.append(("xgboost", XGBClassifier()))
models.append(("SVM", SVC(probability = True)))
models.append(("Random Forest", RandomForestClassifier()))
models.append(("Random Forest", DecisionTreeClassifier()))
models.append(("Logistic Regression", LogisticRegression()))
models.append(("Ada Boost", AdaBoostClassifier(base_estimator = RandomForestClassifier(n_estimators = 50))))

In [39]:
Model(df, models, thrsold = 0.5, grid = False, cv = False)

C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:01:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(                  Algo  Accuracy  Precision    Recall  F1_score       Auc
 0              xgboost  0.993590   0.989337  0.997951  0.993626  0.993584
 1                  SVM  0.998846   1.000000  0.997695  0.998846  0.998848
 2        Random Forest  1.000000   1.000000  1.000000  1.000000  1.000000
 3        Random Forest  1.000000   1.000000  1.000000  1.000000  1.000000
 4  Logistic Regression  0.986218   0.979180  0.993598  0.986336  0.986208
 5            Ada Boost  0.999936   1.000000  0.999872  0.999936  0.999936,
                   Algo  Accuracy  Precision    Recall  F1_score       Auc
 0              xgboost  0.988269   0.981804  0.995006  0.988361  0.988261
 1                  SVM  0.990962   0.990031  0.991932  0.990981  0.990960
 2        Random Forest  0.993654   0.990084  0.997311  0.993684  0.993650
 3        Random Forest  0.994231   0.994617  0.993853  0.994235  0.994231
 4  Logistic Regression  0.979423   0.967766  0.991932  0.979700  0.979409
 5            Ada Boost 

Making a Predictive System

In [64]:
# Bulding a predictive system

for i in range(1,1000):
    prediction = Y_test[i]


    if prediction == 1:
        print(f"The News {i} is Real")
        print("\n")
        print("*" * 50)
    else:
        print(f"The News {i} is Fake")
        print("\n")
        print("*" * 50)


The News 1 is Fake


**************************************************
The News 2 is Real


**************************************************
The News 3 is Fake


**************************************************
The News 4 is Real


**************************************************
The News 5 is Real


**************************************************
The News 6 is Fake


**************************************************
The News 7 is Fake


**************************************************
The News 8 is Fake


**************************************************
The News 9 is Fake


**************************************************
The News 10 is Real


**************************************************
The News 11 is Fake


**************************************************
The News 12 is Fake


**************************************************
The News 13 is Real


**************************************************
The News 14 is Real


**************************************